### Sentiment Analysis for twitter tweets - experiment with different algorithms

In [19]:
import pandas as pd
import numpy as np
from scipy.special import softmax

import os
from tqdm import tqdm

In [20]:
filepath = '../../data/Tweets/tweets_complete'

In [21]:
df = pd.read_csv(f'{filepath}/2014-01-02.csv')

In [22]:
df.head()

,Unnamed: 0,user,fullname,timestamp,text
0,1568219,ProjectCoin,Project Coin,2014-01-02,"LIVE: Profit = $40,782.59 (4.39 %). BUY B1148...."
1,1568613,indobtc,Indonesia Bitcoin,2014-01-02,Harga #Bitcoin Per Tanggal 02-01-2014 08:00:04...
2,1568614,bitcoinstuffUK,BitCoin Stuff,2014-01-02,1 BTC = 485.00 GBP at 13:12 January/2/2014 #Bi...
3,1568616,PanamaTicker,PTY TICKER,2014-01-02,2014-01-02 08:11:22 Panamabitcoins ticker is 7...
4,1568617,InfoGerejaBdg,Info Gereja Bandung,2014-01-02,RT @mega_rure\nDoa Fajar @BFABandung untuk men...


In [23]:
files = os.listdir(filepath)
files[:10]

['2014-01-02.csv',
 '2014-01-03.csv',
 '2014-01-04.csv',
 '2014-01-05.csv',
 '2014-01-06.csv',
 '2014-01-07.csv',
 '2014-01-08.csv',
 '2014-01-09.csv',
 '2014-01-10.csv',
 '2014-01-11.csv']

### Experiment with only 10 to get best implementation

In [25]:
dfs = [pd.read_csv(f'{filepath}/{i}', engine='python') for i in files[:10]]

In [26]:
len(dfs)

10

### Sentiment analysis using VADER

**Stop words, tokenization, stemming & lemmatization are not required for VADER**

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [44]:
# Only required preprocess (remove username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(' '):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return ' '.join(new_text)

def calculate_sentiment(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)        
    return sentiment_dict['neg'], sentiment_dict['neu'], sentiment_dict['pos'], sentiment_dict['compound']

In [25]:
# Test sentiment
test_sentence = 'ANN is like our brain; millions and billions of cells — called neurons, which processes information in the form of electric signals. Similarly, in ANN, the network structure has an input layer, a hidden layer, and the output layer. It is also called Multi-Layer Perceptron as it has multiple layers. The hidden layer is known as a “distillation layer” that distils some critical patterns from the data/information and passes it onto the next layer. It then makes the network quicker and more productive by distinguishing the data from the data sources, leaving out the excess data.'

In [45]:
calculate_sentiment(test_sentence)

(0.023, 0.951, 0.025, 0.0516)

The values provided by the VADER algorithm are follows:
* neg: negative score
* neu: neutral score
* pos: positive score
* compound: sum of above three and normalized between -1 and +1
  * The more closer the value is to +1, the higher the positivity of the text.

In [31]:
dfs[0].iloc[0]['text']

'LIVE: Profit = $40,782.59 (4.39 %). BUY B1148.83 @ $825.80 (#BitStamp). SELL @ $835.00 (#MtGox) #bitcoin #btc\r'

In [40]:
dfs[0].shape[0]

500

In [48]:
def get_sentiments():
    for i, df in tqdm(enumerate(dfs)):
        print(f'Currently at df: {i+1}')
        negative_scores = []
        positive_scores = []
        neutral_scores = []
        compound_scores = []
        
        for j in tqdm(range(df.shape[0])):
            neg, neu, pos, compound = calculate_sentiment(df.iloc[j]['text'])
            negative_scores.append(neg)
            positive_scores.append(neu)
            neutral_scores.append(pos)
            compound_scores.append(compound)
            
        df['negative_score'] = negative_scores
        df['positive_score'] = positive_scores
        df['neutral_score'] = neutral_scores
        df['compound_score'] = compound_scores

In [49]:
get_sentiments()

0it [00:00, ?it/s]
  2%|██▊                                                                                                                                                         | 9/500 [00:00<00:05, 85.66it/s]

Currently at df: 1



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 66.65it/s]
1it [00:07,  7.51s/it]
  2%|██▍                                                                                                                                                         | 8/500 [00:00<00:06, 73.90it/s]

Currently at df: 2



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 68.71it/s]
2it [00:14,  7.44s/it]
  1%|██▏                                                                                                                                                         | 7/500 [00:00<00:07, 68.28it/s]


...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 70.06it/s]
8it [00:57,  7.13s/it]
  2%|██▍                                                                                                                                                         | 8/500 [00:00<00:06, 70.70it/s]

Currently at df: 9



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:09<00:00, 55.22it/s]
9it [01:06,  7.71s/it]
  1%|██▏                                                                                                                                                         | 7/500 [00:00<00:07, 68.94it/s]

Currently at df: 10



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 65.04it/s]
10it [01:13,  7.38s/it]


In [50]:
dfs[0]

,Unnamed: 0,user,fullname,timestamp,text,sentiment,negative_score,positive_score,neutral_score,compound_score
0,1568219,ProjectCoin,Project Coin,2014-01-02,"LIVE: Profit = $40,782.59 (4.39 %). BUY B1148....",0,0.00,0.847,0.153,0.4404
1,1568613,indobtc,Indonesia Bitcoin,2014-01-02,Harga #Bitcoin Per Tanggal 02-01-2014 08:00:04...,0,0.00,1.000,0.000,0.0000
2,1568614,bitcoinstuffUK,BitCoin Stuff,2014-01-02,1 BTC = 485.00 GBP at 13:12 January/2/2014 #Bi...,0,0.00,1.000,0.000,0.0000
3,1568616,PanamaTicker,PTY TICKER,2014-01-02,2014-01-02 08:11:22 Panamabitcoins ticker is 7...,0,0.00,1.000,0.000,0.0000
4,1568617,InfoGerejaBdg,Info Gereja Bandung,2014-01-02,RT @mega_rure\nDoa Fajar @BFABandung untuk men...,0,0.13,0.751,0.119,-0.0772
...,...,...,...,...,...,...,...,...,...,...
495,1575277,mtgoxtrades,Mt.GoxTrades,2014-01-02,Fri Jan 3 00:28:59 2014 - Traded 0.63293284 B...,0,0.00,1.000,0.000,0.0000
496,1575265,mtgoxtrades,Mt.GoxTrades,2014-01-02,Fri Jan 3 00:32:13 2014 - Traded 0.27059347 B...,0,0.00,1.000,0.000,0.0000
497,1575290,mtgoxtrades,Mt.GoxTrades,2014-01-02,Fri Jan 3 00:12:38 2014 - Traded 2 BTC at 860...,0,0.00,1.000,0.000,0.0000
498,1575292,BTCNewsTicker,BTCNewsTicker,2014-01-02,#BitCoin Marketvalue\nBitCoin.de Last Transact...,0,0.00,1.000,0.000,0.0000


### Sentiment analysis using Twitter-roBERTa-base

**BERT model uses all information from a sentence, therefore preprocessing isn't needed**

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

In [4]:
# Only required preprocess (remove username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(' '):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return ' '.join(new_text)

In [14]:
MODEL = f'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Save models
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)

In [29]:
text = preprocess('Covid cases are increasing fast!')
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f'{i+1}) {l} {np.round(float(s), 4)}')

0.22867917
1) Negative 0.7236
2) Neutral 0.2287
3) Positive 0.0477


Similar values produced without "compound"

In [33]:
ranking

array([0, 1, 2], dtype=int64)

In [38]:
def calculate_sentiment(sentence):
    sentiment_dict = {}
    encoded_input = tokenizer(preprocess(sentence), return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(len(scores)):
        label = config.id2label[ranking[i]]
        score = scores[ranking[i]]
        sentiment_dict[label] = score
        
    return sentiment_dict['Negative'], sentiment_dict['Neutral'], sentiment_dict['Positive']

In [39]:
calculate_sentiment(preprocess(text))

(0.7235764, 0.22867917, 0.047744352)

In [42]:
def get_sentiments():
    for i, df in tqdm(enumerate(dfs)):
        print(f'Currently at df: {i+1}')
        negative_scores = []
        positive_scores = []
        neutral_scores = []
        
        for j in tqdm(range(df.shape[0])):
            neg, neu, pos, = calculate_sentiment(df.iloc[j]['text'])
            negative_scores.append(neg)
            positive_scores.append(neu)
            neutral_scores.append(pos)
            
        df['negative_score'] = negative_scores
        df['positive_score'] = positive_scores
        df['neutral_score'] = neutral_scores

In [43]:
get_sentiments()

0it [00:00, ?it/s]
  0%|                                                                                                                                                                                      | 0/500 [00:00<?, ?it/s]

Currently at df: 1



 16%|██████████████████████████▉                                                                                                                                                  | 78/500 [00:19<01:49,  3.84it/s]


 34%|███████████████████████████████████████████████████████████▏                                                                                                                | 172/500 [00:39<01:05,  5.00it/s]


 53%|███████████████████████████████████████████████████████████████████████████████████████████▏                                                                                | 265/500 [00:55<00:47,  4.92it/s]


...

 48%|██████████████████████████████████████████████████████████████████████████████████▉                                                                                         | 241/500 [00:57<01:26,  3.00it/s]


 64%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                              | 319/500 [01:15<00:33,  5.39it/s]


 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                  | 399/500 [01:33<00:22,  4.53it/s]


 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌    | 487/500 [01:51<00:02,  4.85it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [01:53<00:00,  4.39it/s]
10it [19:15, 115.54s/it]


Transformers are much slower than VADER. Since more faster execution is necessary, VADER will be the choice of NLP sentiment analyser.